In [1]:
from pib import PIB
import pandas as pd

In [2]:
pib = PIB('490dcd21-44b2-fef6-0ade-e6431a8c8fb9')

entidades = (pd.read_parquet("../../data/catalogos/estados/estados.parquet")
            #  .sort_values(by="nom_agee")
             ).nom_agee.apply(lambda x: x.capitalize())

In [3]:
def pib_real():
    df = pd.DataFrame()

    for entidad in entidades:
        print(entidad)
        temp = pib.get_data_estatal(grupo = "estado", entidad=entidad)
        temp["entidad"] = entidad
        df = pd.concat([df,temp])

    return df

df_ = pib_real()

Baja california
Baja california sur
Campeche
Colima
Guerrero
Jalisco
Nayarit
Oaxaca
Quintana roo
Sinaloa
Sonora
Veracruz de ignacio de la llave
Yucatán
Aguascalientes
Coahuila de zaragoza
Chiapas
Chihuahua
Ciudad de méxico
Durango
Guanajuato
Hidalgo
México
Michoacán de ocampo
Morelos
Nuevo león
Puebla
Querétaro
San luis potosí
Tabasco
Tamaulipas
Tlaxcala
Zacatecas


In [7]:
def pib_nom():
    df = pd.DataFrame()

    for entidad in entidades:
        print(entidad)
        temp = pib.get_data_estatal(indicador="PIB anual a precios corrientes", grupo = "estado", entidad=entidad)
        temp["entidad"] = entidad
        df = pd.concat([df,temp])

    return df

df_nom = pib_nom()

Baja california
Baja california sur
Campeche
Colima
Guerrero
Jalisco
Nayarit
Oaxaca
Quintana roo
Sinaloa
Sonora
Veracruz de ignacio de la llave
Yucatán
Aguascalientes
Coahuila de zaragoza
Chiapas
Chihuahua
Ciudad de méxico
Durango
Guanajuato
Hidalgo
México
Michoacán de ocampo
Morelos
Nuevo león
Puebla
Querétaro
San luis potosí
Tabasco
Tamaulipas
Tlaxcala
Zacatecas


In [12]:
df_.to_excel("../../data/pib/pib_realv2.xlsx")
df_nom.to_excel("../../data/pib/pib_nominalv2.xlsx")

In [9]:
df_ = pd.read_excel("../../data/pib/pib_realv2.xlsx", dtype={"periodo":str}).set_index("periodo")

df_nom = pd.read_excel("../../data/pib/pib_nominalv2.xlsx", dtype={"periodo":str}).set_index("periodo")

In [11]:
def ranking(df, sector, year:int):
    ranking = pd.DataFrame()
    year = str(year)
    for entidad in entidades:
        temp = df[df["entidad"] == entidad].loc[[year]]
        temp["entidad"] = entidad
        ranking = pd.concat([ranking,temp])
    
    ranking = ranking.reset_index(drop=True)
    ranking = ranking.set_index("entidad")
    ranking = ranking[[sector]]
    ranking["rank"] = (ranking.rank(method="first", ascending=False))
    ranking = ranking.sort_values(by="rank")

    X = ranking[sector].sum()

    ranking["% nacional"] = (ranking[sector]/X)*100

    return ranking

ranking(df_nom, "Total", 2022)

,Total,rank,% nacional
entidad,,,
Ciudad de méxico,4287768.210,1.0,14.558085
México,2647537.325,2.0,8.989076
Nuevo león,2356106.867,3.0,7.999594
Jalisco,2212640.428,4.0,7.512488
Guanajuato,1374749.364,5.0,4.667630
Veracruz de ignacio de la llave,1251659.524,6.0,4.249709
Coahuila de zaragoza,1149123.673,7.0,3.901573
Baja california,1146704.230,8.0,3.893358
Chihuahua,1125154.076,9.0,3.820190


In [ ]:
def variacion(df, sector:str, year: int):
    df_f = pd.DataFrame()

    yeari = str(year - 1)
    yearf = str(year)

    for entidad in entidades:
        temp = df[df["entidad"] == entidad].loc[[yeari, yearf]]
        temp["entidad"] = entidad
        temp = temp[["entidad",sector]]
        temp["cambio %"] = (temp[sector].pct_change())*100
        # temp = temp.dropna()
        df_f = pd.concat([df_f,temp])
    

    return df_f

variacion(df_, "Total industria manufacturera", 2022)

In [ ]:
def  var_nac():
    df = (variacion(df_, "Total industria manufacturera", 2022)
     .reset_index())
    
    sum_2021 = df[df["periodo"] == "2021"]["Total industria manufacturera"].sum()
    sum_2022 = df[df["periodo"] == "2022"]["Total industria manufacturera"].sum()

    var = ((sum_2022 - sum_2021)/sum_2021)*100

    return var

var_nac()

In [ ]:
class INFO_PIB:

    def __init__(self) -> None:
        pass


    def _ranking(self, df, sector, year):
        ranking = pd.DataFrame()
        year = str(year)
        for entidad in entidades:
            temp = df[df["entidad"] == entidad].loc[[year]]
            temp["entidad"] = entidad
            ranking = pd.concat([ranking,temp])
        
        ranking = ranking.reset_index(drop=True)
        ranking = ranking.set_index("entidad")
        ranking = ranking[[sector]]
        ranking["rank"] = (ranking.rank(method="first", ascending=False))
        ranking = ranking.sort_values(by="rank")

        X = ranking[sector].sum()

        ranking["% nacional"] = (ranking[sector]/X)*100

        return ranking
    

    def _variacion(self, df, sector, year):
        df_f = pd.DataFrame()
        yeari = str(year - 1)
        yearf = str(year)

        for entidad in entidades:
            temp = df[df["entidad"] == entidad].loc[[yeari, yearf]]
            temp["entidad"] = entidad
            temp = temp[["entidad",sector]]
            temp["cambio %"] = (temp[sector].pct_change())*100
            temp = temp.dropna()
            df_f = pd.concat([df_f,temp])
        

        return df_f
    
    
    def get_ranking(self,
                    df: pd.DataFrame,
                    sector: str|list = "Total", 
                    year: int = 2022,
                    estado: str = "Aguascalientes"
                    ):
        
        ranking = self._ranking(df, sector, year)
        var = ranking.loc[estado][0]
        rank = ranking.loc[estado][1]
        nac =  round(ranking.loc[estado][2],2)

        return [var, rank, nac]
    

    def get_var(self, 
                df: pd.DataFrame,
                sector: str = "Total",
                year: int = 2022,
                estado: str = "Aguascalientes",
                ):
        
        var = self._variacion(df, sector, year).set_index("entidad")
        var = round(var.loc[estado][1], 2)

        return var


info = INFO_PIB()

In [ ]:
info.get_var(df_)